In [ ]:
import os

from dateutil.parser import parse
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd

from src.data_handler import DataHandler
from src.analysis_logic import FitMethods, AnalysisLogic
from src.helper_functions import format_exponent_as_str

plt.style.use('seaborn')
plt.rcParams['image.cmap'] = sns.color_palette("mako", as_cmap=True)

In [ ]:
tip_2R16 = DataHandler(measurement_folder="20221124_FR0612-F2-2R16_PCB_UHVstage")
analysis = AnalysisLogic()

In [ ]:
files = [file for file in os.listdir(tip_2R16.data_folder_path) if file.endswith(".csv") and "long" not in file]

fig, ax = plt.subplots(nrows=len(files) - 1, sharex=True)

for idx, file in enumerate(files[:-1]):
    df = pd.read_csv(os.path.join(tip_2R16.data_folder_path, file), skiprows=14)
    ax[idx].plot(df["Power (W)"] * 1e6)
    ax[idx].set_title(f"{file}, std={df['Power (W)'].std() * 1e6:.1f} uW, mean={df['Power (W)'].mean() * 1e6:.1f} uW")

fig.tight_layout()
# tip_2R16.save_figures(fig, filename="power_fluctuation_with_laser_setpoint")

In [ ]:
files = [file for file in os.listdir(tip_2R16.data_folder_path) if file.endswith(".csv") and "long" in file]

fig, ax = plt.subplots(nrows=len(files), ncols=2, sharex="col", figsize=(10, 10))

for idx, file in enumerate(files):
    df = pd.read_csv(os.path.join(tip_2R16.data_folder_path, file), skiprows=14)
    df = df.iloc[:8000]
    ax[idx, 0].plot(df["Power (W)"] * 1e6)

    if file == '25mW_long.csv' or file == 'background_long.csv' or file == "20mW_long.csv":
        fit_function = FitMethods.linear
        label = "Linear"
    else:
        fit_function = FitMethods.decayexponential
        label = "Exp. decay"

    fit_x, fit_y, result = analysis.perform_fit(df.index, df["Power (W)"] * 1e6, fit_function=fit_function)
    ax[idx, 0].plot(fit_x, fit_y, label=label)
    ax[idx, 0].legend()

    power = file.split("_")[0]
    residual_sum_norm = np.sum(np.absolute(result.residual)) / len(df)
    ax[idx, 0].set_title(f"{power}, " + r"$\chi_{\nu}^{2}$ = " + f"{result.redchi:.2f}")

    #x_pre = np.arange(0, 15000)
    #pred = result.eval(x=x_pre)
    #ax[idx, 0].plot(x_pre, pred, "--")
    
    diff = np.diff(fit_y)
    ax[idx, 1].plot(fit_x[:-1], diff)
    conv = fit_x[np.where(np.abs(diff) < 1e-5)]
    if conv.any():
        ax[idx, 1].axvline(conv[0], color="C1", linestyle="--")
        ax[idx, 1].set_title(f"differential convergence = {int(conv[0])} s")
    

ax[-1, 0].set_xlabel("Time (s)")
ax[-1, 1].set_xlabel("Time (s)")

fig.tight_layout()
tip_2R16.save_figures(fig, filename="long_term_power_fluctuation_comparison")

In [ ]:
from scipy.signal import savgol_filter

In [ ]:
files = [file for file in os.listdir(tip_2R16.data_folder_path) if file.endswith(".csv") and "long" in file]

fig, ax = plt.subplots(figsize=(7, 5))

for idx, file in enumerate(files):
    df = pd.read_csv(os.path.join(tip_2R16.data_folder_path, file), skiprows=14)
    df = df.iloc[:8000]
    
    if file == '25mW_long.csv' or file == 'background_long.csv' or file == "20mW_long.csv":
        fit_function = FitMethods.linear
    else:
        fit_function = FitMethods.decayexponential
        
    fit_x, fit_y, result = analysis.perform_fit(df.index, df["Power (W)"] * 1e6, fit_function=fit_function)
    power = file.split("_")[0]
    ax.plot(savgol_filter(df["Power (W)"] * 1e6 / fit_y[-1], 101, 2), linewidth=1, label=f"{power}")
    
ax.set_ylabel(r"$\frac{P(t)}{P(t=8000)}$")
ax.set_xlabel(r"$t\ (s)$")
ax.legend()

tip_2R16.save_figures(fig, filename="long_term_power_fluctuation_normalized_filtered_comparison")

In [ ]:
file = "100mW_VL.csv"

fig, ax = plt.subplots(figsize=(7, 5))

df = pd.read_csv(os.path.join(tip_2R16.data_folder_path, file), skiprows=14)

fit_function = FitMethods.decayexponential
fit_x, fit_y, result = analysis.perform_fit(df.index, df["Power (W)"] * 1e6, fit_function=fit_function)

ax.plot(df["Power (W)"] * 1e6, linewidth=1)
ax.plot(fit_x, fit_y, label=fit_function.value)

ax.set_ylabel(r"P (t)")
ax.set_xlabel(r"t (s)")
ax.set_title(file)
ax.legend()

print(result.fit_report())

Keep laser at 100 mW, adjust ND filter to get 40 uW at beamsplitter (160 uW at tip)